**Esta demo esta fuera de uso**. Sus contenidos demo están cubiertos dentro de la demo 7_ajuste_de_datos/distribucion_lateral.ipynb. 

# Región de confianza no Gaussiana 

Usando la función de costo / verosimilitud

In [ ]:
%matplotlib notebook

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
from scipy.stats import chi2
from scipy.optimize import minimize

In [ ]:
def get_ellipse(center, cova, nsigma=1):
    L = np.linalg.cholesky(cova)
    t = np.linspace(0, 2 * np.pi, 1000)
    circulo = np.column_stack([np.cos(t), np.sin(t)])
    elipse =  nsigma * circulo @ L.T + center
    return elipse.T

## Datos

In [ ]:
xdata = np.array([191, 263, 309])  # distancia en metros
ydata = np.array([33, 19, 11])     # número de partículas medidas

## Modelo

In [ ]:
def modelo(x, theta):
    S0 = theta[0]
    beta = theta[1]
    x0 = 250
    return S0 * np.power(x/x0, -beta)

## Función de costo

$J(S_{0}, \beta) = 2 \, \sum_i \left[ (\mu_i(S_{0}, \beta)-y_i) - k_i \log(\mu_i(S_{0}, \beta) / y_i) \right]$

In [ ]:
def funcion_costo(theta, xdata, ydata):
        costo = np.zeros_like(theta[0])
        for (x1, y1) in zip(xdata, ydata):
            mu1 = modelo(x1, theta)
            costo += (mu1-y1) - y1 * np.log(mu1/y1)
        return 2*costo

In [ ]:
J = lambda theta: funcion_costo(theta, xdata, ydata)  

## Estimadores de máxima verosimilitud

In [ ]:
res = minimize(J, x0=(3,0.2))
res

In [ ]:
theta_est = res.x
theta_est

## Valle de la verosimilitud

In [ ]:
S0 = np.linspace(10, 30, 128)
beta = np.linspace(0.1, 4.5, 128)
S0, beta = np.meshgrid(S0, beta)
theta = np.array([S0, beta])
z = J(theta)

In [ ]:
fig1 = plt.figure(figsize=(4,4))
ax1 = fig1.subplots(subplot_kw={"projection": "3d"})
ax1.set_xlabel('$S_0$')
ax1.set_ylabel('β')
ax1.set_zlabel('J($S_0$,β)')
zmin = res.fun
levels = zmin + np.array([0, 1, 4, 9]) 
norm = colors.BoundaryNorm(boundaries=levels, ncolors=len(levels))
ax1.plot_surface(S0, beta, z, cmap=cm.tab10, norm=norm, rstride=1, cstride=1, linewidth=0)

In [ ]:
# fig1.savefig('region_anormal_costo.svg')

## Contorno de niveles

In [ ]:
fig2, ax2 = plt.subplots()
ax2.set_xlabel('$S_0$')
ax2.set_ylabel('β')
levels = zmin + np.array([1, 4, 9]) 
contour = ax2.contour(S0, beta, z, levels, colors=['tab:blue', 'tab:orange', 'tab:green'])

Etiquetas de los niveles de contorno

In [ ]:
fmt = {}
strs = ['1σ', '2σ', '3σ']
for l, s in zip(contour.levels, strs):
    fmt[l] = s
ax2.clabel(contour, fmt=fmt)
fmt

In [ ]:
cova = 2 * res.hess_inv
error = np.sqrt(np.diagonal(cova))
error

In [ ]:
rho = cova[0][1]/(error[0]*error[1])
rho

In [ ]:
elipse_1sigma = get_ellipse(theta_est, cova, nsigma=1)
ax2.plot(*elipse_1sigma, ls='--')
elipse_2sigma = get_ellipse(theta_est, cova, nsigma=2)
ax2.plot(*elipse_2sigma, ls='--')
elipse_3sigma = get_ellipse(theta_est, cova, nsigma=3)
ax2.plot(*elipse_3sigma, ls='--')

In [ ]:
ax2.plot(*theta_est, 'o')

In [ ]:
# fig2.savefig('region_anormal.svg')

## Bondad del ajuste

In [ ]:
# Jmin = res.fun
# Jmin

In [ ]:
# ndof = len(xdata) - len(theta_est)
# ndof

In [ ]:
# pvalor = chi2.sf(Jmin, df=ndof)
# pvalor